# Variables

In [1]:
ga_input = '/grid/home/nbourgeois/wip/lau_lyn_cdo/gaa.pep.fas' #sequence pep.fas Input
tissue = 'Muscle_Skeletal' # Tissue type for CoCoPuts
filename = 'bai_1'

# Imports


In [2]:
import os
cwd = os.getcwd()

## Import the code for codon opt
import sys
sys.path.insert(1, '/grid/home/nbourgeois/codonOpt')
# from run_pygad import *
from general_functions import *
from dyn_prog import *

import os
cwd = os.getcwd()
pygad_loc = '/grid/home/nbourgeois/codonOpt/'
os.chdir(pygad_loc)
import pygad
import numpy as np
from general_functions import *
from metrics import *
import time
import glob
import multiprocessing
from general_functions import *
from metrics import *
import shutil
import sys
os.chdir(cwd)

Using TensorFlow backend.


# Data

In [3]:
(keys, seqs) = readFasta(ga_input)
if len(seqs) == 1:
    aa_seq=str(seqs[0])

In [4]:
def predictSites_multi( fastaIds,fastaSeqs,threshold=.1):

    gpu0_avail = gpu_memory_usage(0)
    gpu1_avail = gpu_memory_usage(1)
    if gpu0_avail + gpu1_avail > 30000:
        print('Out of memory\n')
        return(-1)
    if gpu0_avail < gpu1_avail:
        gpu="0"
    else:
        gpu="1"
    
    os.environ["CUDA_VISIBLE_DEVICES"]=gpu

    from keras.models import load_model
    import tensorflow as tf
    from tensorflow import keras

    results = {}
    gpus =tf.config.experimental.list_physical_devices('GPU')

    try: 
        tf.config.experimental.set_virtual_device_configuration(gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
        
        context = 10000
        keras.backend.set_learning_phase(0)
        paths = ('models/spliceai{}.h5'.format(x) for x in range(1, 6))
        models = [load_model(resource_filename('spliceai', x), compile=False ) for x in paths]
        for seq_id, seq in zip(fastaIds,fastaSeqs):

            x = one_hot_encode('N'*(context//2) + str(seq) + 'N'*(context//2))[None, :]
            y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
            acceptor_prob = y[0, :, 1]
            donor_prob = y[0, :, 2]
            coord = list( range(1, len( donor_prob )+1 ) )
                    
            # custom metric
#             results[seq_id] = (len(acceptor_prob[acceptor_prob > threshold]) + len(donor_prob[donor_prob > threshold]))/2
            acceptor_count = 0
            donor_count = 0
            # for i in coord:
            #     if donor_prob[i] > threshold :
            #         donor_count += donor_prob[i]
            #     if acceptor_prob[i] > threshold and donor_count > 0:
            #         acceptor_count += acceptor_prob[i]
            # custom metric
            results[seq_id] = {'Coord':coord,'Donor':donor_prob,'Acceptor':acceptor_prob}
            # results.append(acceptor_count * donor_count)
        return(results)
    except tf.errors as e:
        # Splice AI custome sequeence prediction
        print(e,'\n')
        return(-1)

# Genetic Algorithm

In [5]:
%%time
optimizer = Optimizer(aa_seq,tissues=tissue )
optimized_seq = optimizer.optimize()
get_bai(optimized_seq,optimizer.bai_weight_dict[tissue])

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

0.9169638283374133

In [6]:
optimizer.codon_chains[-1]

{'TAA': ['ATG',
  'GGC',
  'GTG',
  'CGC',
  'CAC',
  'CCC',
  'CCC',
  'TGC',
  'AGC',
  'CAC',
  'CGC',
  'CTG',
  'CTG',
  'GCC',
  'GTG',
  'TGT',
  'GCC',
  'CTG',
  'GTG',
  'TCC',
  'CTG',
  'GCC',
  'ACC',
  'GCC',
  'GCC',
  'CTG',
  'CTG',
  'GGC',
  'CAC',
  'ATC',
  'CTG',
  'CTG',
  'CAC',
  'GAC',
  'TTC',
  'CTG',
  'CTG',
  'GTG',
  'CCC',
  'CGG',
  'GAG',
  'CTG',
  'TCC',
  'GGC',
  'TCC',
  'TCC',
  'CCT',
  'GTG',
  'CTG',
  'GAG',
  'GAG',
  'ACC',
  'CAC',
  'CCC',
  'GCC',
  'CAC',
  'CAG',
  'CAG',
  'GGG',
  'GCC',
  'AGC',
  'CGC',
  'CCC',
  'GGC',
  'CCC',
  'CGG',
  'GAC',
  'GCC',
  'CAG',
  'GCC',
  'CAC',
  'CCC',
  'GGC',
  'CGC',
  'CCC',
  'CGG',
  'GCC',
  'GTG',
  'CCC',
  'ACC',
  'CAG',
  'TGT',
  'GAC',
  'GTG',
  'CCC',
  'CCC',
  'AAC',
  'AGC',
  'CGC',
  'TTC',
  'GAC',
  'TGT',
  'GCC',
  'CCC',
  'GAC',
  'AAG',
  'GCC',
  'ATC',
  'ACC',
  'CAG',
  'GAG',
  'CAG',
  'TGT',
  'GAG',
  'GCC',
  'CGG',
  'GGC',
  'TGC',
  'TGC',
  'TAC',
  '

In [7]:
%%time
optimizer = Optimizer(aa_seq,tissues=tissue )
optimizer.depth=2
optimized_seq2 = optimizer.optimize()
get_bai(optimized_seq,optimizer.bai_weight_dict[tissue])

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

0.9169638283374133

In [8]:
( optimized_seq ==optimized_seq2)

True

In [9]:

len( optimized_seq2)

2859

In [10]:
# %%time
# optimizer = Optimizer(aa_seq,tissues=tissue )
# optimizer.depth=3
# optimized_seq = optimizer.optimize()
# get_bai(optimized_seq,optimizer.bai_weight_dict[tissue])

In [11]:
# %%time
# optimizer = Optimizer(aa_seq,tissues=tissue )
# optimizer.depth=4
# optimized_seq = optimizer.optimize()
# get_bai(optimized_seq,optimizer.bai_weight_dict[tissue])

In [12]:
optimized_seq2 = optimizer.trim(307)
get_bai(optimized_seq2,optimizer.bai_weight_dict[tissue])

102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 

0.9156710688492328

In [13]:
# optimizer = Optimizer(aa_seq, tissues=tissue, ntissues='Liver')
# optimized_seq2 = optimizer.optimize()
# optimizer.bai_weight_dict.keys()

In [14]:
sps_results3 = predictSites_multi(['seq','seq2'],[optimized_seq,optimized_seq2],.3)


2022-08-11 18:03:31.150416: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-08-11 18:03:31.167558: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-PCIE-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:d7:00.0
2022-08-11 18:03:31.167808: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2022-08-11 18:03:31.170128: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-08-11 18:03:31.172056: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2022-08-11 18:03:31.172398: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10.0
2022-08-11 18:03:31.174854: I tensorflow/stream_executor/platfo

In [15]:
thresh=.01
df_all = pd.DataFrame(sps_results3['seq'])
df_t = df_all.loc[(df_all['Acceptor']>thresh) | (df_all['Donor']>thresh)]
df_t

,Coord,Donor,Acceptor
306,307,0.012696,0.000044
486,487,0.000115,0.029419
1436,1437,0.014637,0.000026
2046,2047,0.000041,0.010747
2145,2146,0.000034,0.014789
2285,2286,0.016797,0.000212
2481,2482,0.000025,0.019808
2786,2787,0.000001,0.019972


In [16]:
thresh=.01
df_all = pd.DataFrame(sps_results3['seq2'])
df_t = df_all.loc[(df_all['Acceptor']>thresh) | (df_all['Donor']>thresh)]
df_t

,Coord,Donor,Acceptor
486,487,0.000128,0.022968
1436,1437,0.014156,0.000025
2046,2047,0.000039,0.010895
2145,2146,0.000035,0.014257
2285,2286,0.015246,0.000204
2481,2482,0.000024,0.018587
2786,2787,0.000001,0.019634


In [17]:
# get_bai(optimized_seq,optimizer.bai_weight_dict['Liver']),get_bai(optimized_seq,optimizer.bai_weight_dict['Heart_Atrial_Appendage'])
# get_bai(optimized_seq2,optimizer.bai_weight_dict['Liver']),get_bai(optimized_seq2,optimizer.bai_weight_dict['Heart_Atrial_Appendage'])
list1 = ['a','b','c']
list1[0:4]

['a', 'b', 'c']

In [18]:
import itertools

list1 = ['a','b','c']
list2 = ['d','e','f']
list3 = ['g','h','i']
optimizer.gene_space[1:3]
for codons in [list(x) for x in list(itertools.product(['ATG'],*[list1,list2]))]:
    print(codons)
    print('_____')

['ATG', 'a', 'd']
_____
['ATG', 'a', 'e']
_____
['ATG', 'a', 'f']
_____
['ATG', 'b', 'd']
_____
['ATG', 'b', 'e']
_____
['ATG', 'b', 'f']
_____
['ATG', 'c', 'd']
_____
['ATG', 'c', 'e']
_____
['ATG', 'c', 'f']
_____


In [19]:
merged = list(itertools.chain(*codons))
merged = list(itertools.chain(*merged))
merged = list(itertools.chain(*merged))
merged

['A', 'T', 'G', 'c', 'f']